[![AWS SDK for pandas](_static/logo.png "AWS SDK for pandas")](https://github.com/aws/aws-sdk-pandas)

# 34 - Glue Data Quality

 AWS Glue Data Quality helps you evaluate and monitor the quality of your data.

## Create test data

First, let's start by creating test data, writing it to S3, and registering it in the Glue Data Catalog.

In [ ]:
import awswrangler as wr
import pandas as pd

glue_database = "aws_sdk_pandas"
glue_table = "my_glue_table"
path = "s3://BUCKET_NAME/my_glue_table/"

df = pd.DataFrame({"c0": [0, 1, 2], "c1": [0, 1, 2], "c2": [0, 0, 1]})
wr.s3.to_parquet(df, path, dataset=True, database=glue_database, table=glue_table)

## Start with recommended data quality rules

AWS Glue Data Quality can recommend a set of data quality rules so you can get started quickly.

Note: Running Glue Data Quality recommendation and evaluation tasks requires an IAM role. This role must trust the Glue principal and allow permissions to various resources including the Glue table and the S3 bucket where your data is stored. Moreover, data quality IAM actions must be granted. To find out more, check [Authorization](https://docs.aws.amazon.com/glue/latest/dg/data-quality-authorization.html).

In [2]:
first_ruleset = "ruleset_1"
iam_role_arn = "arn:aws:iam::..." # IAM role assumed by the Glue Data Quality job to access resources

wr.data_quality.create_recommendation_ruleset(  # Creates a recommended ruleset
    name=first_ruleset,
    database=glue_database,
    table=glue_table,
    iam_role_arn=iam_role_arn,
    number_of_workers=2,
)

rule_type parameter       expression
0      RowCount      None  between 1 and 6
1    IsComplete      "c0"             None
2    Uniqueness      "c0"           > 0.95
3  ColumnValues      "c0"             <= 2
4    IsComplete      "c1"             None
5    Uniqueness      "c1"           > 0.95
6  ColumnValues      "c1"             <= 2
7    IsComplete      "c2"             None
8  ColumnValues      "c2"             <= 1

## Run a data quality task

In [3]:
wr.data_quality.evaluate_ruleset(
    name=first_ruleset,
    iam_role_arn=iam_role_arn,
    number_of_workers=2,
)

Name               Description Result  \
0  Rule_1  RowCount between 1 and 6   PASS   
1  Rule_2           IsComplete "c0"   PASS   
2  Rule_3    Uniqueness "c0" > 0.95   PASS   
3  Rule_4    ColumnValues "c0" <= 2   PASS   
4  Rule_5           IsComplete "c1"   PASS   
5  Rule_6    Uniqueness "c1" > 0.95   PASS   
6  Rule_7    ColumnValues "c1" <= 2   PASS   
7  Rule_8           IsComplete "c2"   PASS   
8  Rule_9    ColumnValues "c2" <= 1   PASS   

                                            ResultId  
0  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
1  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
2  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
3  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
4  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
5  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
6  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
7  dqresult-eedff508646c69c07cff042b839e6cc1f492e717  
8  dqresult-eedff508646c69c07cff042b839e6cc1f492e717

## Create ruleset from Data Quality Definition Language definition

Recommended rulesets are not perfect and you are likely to want to modify them or create your own.

The Data Quality Definition Language (DQDL) is a domain specific language that you can use to define Data Quality rules. For the full syntax reference, see [DQDL](https://docs.aws.amazon.com/glue/latest/dg/dqdl.html).

In [4]:
second_ruleset = "ruleset_2"

dqdl_rules = (
    "Rules = ["
    "RowCount between 1 and 6,"
    'IsComplete "c0",'
    'Uniqueness "c0" > 0.95,'
    'ColumnValues "c0" <= 2,'
    'IsComplete "c1",'
    'Uniqueness "c1" > 0.95,'
    'ColumnValues "c1" <= 2,'
    'IsComplete "c2",'
    'ColumnValues "c2" <= 1'
    "]"
)

wr.data_quality.create_ruleset(
    name=second_ruleset,
    database=glue_database,
    table=glue_table,
    dqdl_rules=dqdl_rules,
)

## Create or update a ruleset from a data frame

AWS SDK for pandas also allows you to create or update a ruleset from a pandas data frame.

In [7]:
df_rules = pd.DataFrame({
    "rule_type": ["RowCount", "ColumnCorrelation", "Uniqueness"],
    "parameter": [None, '"c0" "c1"', '"c0"'],
    "expression": ["between 2 and 8", "> 0.8", "> 0.95"],
})

wr.data_quality.update_ruleset(
    name=second_ruleset,
    df_rules=df_rules,
    mode="upsert",  # update existing or insert new rules to the ruleset
)

wr.data_quality.get_ruleset(name=second_ruleset)

rule_type  parameter       expression
0         IsComplete       "c0"             None
1       ColumnValues       "c0"             <= 2
2         IsComplete       "c1"             None
3         Uniqueness       "c1"           > 0.95
4       ColumnValues       "c1"             <= 2
5         IsComplete       "c2"             None
6       ColumnValues       "c2"             <= 1
7           RowCount       None  between 2 and 8
8  ColumnCorrelation  "c0" "c1"            > 0.8
9         Uniqueness       "c0"           > 0.95

## Get multiple rulesets

In [9]:
wr.data_quality.get_ruleset(name=[first_ruleset, second_ruleset])

rule_type  parameter       expression    ruleset
0           RowCount       None  between 1 and 6  ruleset_1
1         IsComplete       "c0"             None  ruleset_1
2         Uniqueness       "c0"           > 0.95  ruleset_1
3       ColumnValues       "c0"             <= 2  ruleset_1
4         IsComplete       "c1"             None  ruleset_1
5         Uniqueness       "c1"           > 0.95  ruleset_1
6       ColumnValues       "c1"             <= 2  ruleset_1
7         IsComplete       "c2"             None  ruleset_1
8       ColumnValues       "c2"             <= 1  ruleset_1
0         IsComplete       "c0"             None  ruleset_2
1       ColumnValues       "c0"             <= 2  ruleset_2
2         IsComplete       "c1"             None  ruleset_2
3         Uniqueness       "c1"           > 0.95  ruleset_2
4       ColumnValues       "c1"             <= 2  ruleset_2
5         IsComplete       "c2"             None  ruleset_2
6       ColumnValues       "c2"             <= 1  ruleset_2
7           RowCount       None  between 2 and 8  ruleset_2
8  ColumnCorrelation  "c0" "c1"            > 0.8  ruleset_2
9         Uniqueness       "c0"           > 0.95  ruleset_2